In [31]:
from qiskit import Aer, QuantumCircuit
from qiskit.utils import QuantumInstance
from qiskit_machine_learning.neural_networks import CircuitQNN
from qiskit_machine_learning.algorithms.classifiers import NeuralNetworkClassifier
from qiskit.algorithms.optimizers import COBYLA
from qiskit.circuit.library import RealAmplitudes, ZZFeatureMap

from titanicibmq.titanic_data import *

In [3]:
# Install a pip package in the current Jupyter kernel
import sys
!{sys.executable} -m pip install sklearn

from sklearn.decomposition import PCA

In [4]:
# Load the data

datapath="data/"
data, __ = titanic(datapath=datapath)
data.sample(frac=1)

,Survived,Pclass,Sex,Age,Fare,Embarked,Title,IsAlone,Age*Class
446,1,2,1,0,2,0,2,0,0
74,1,3,0,1,3,0,1,1,3
146,1,3,0,1,0,0,1,1,3
235,0,3,1,1,0,0,2,1,3
512,1,1,0,2,2,0,1,1,2
...,...,...,...,...,...,...,...,...,...
635,1,2,1,1,1,0,2,1,2
743,0,3,0,1,2,0,1,0,3
668,0,3,0,2,1,0,1,1,6
158,0,3,0,1,1,0,1,1,3


In [5]:
X = data.drop("Survived", axis=1).values
pca = PCA(n_components=2)
Xpca = pca.fit_transform(X)

In [6]:
split_ratio = 0.2

X_train = Xpca[int(data.shape[0]*split_ratio):,:]
X_test = Xpca[:int(data.shape[0]*split_ratio),:]

y_train = data.iloc[int(data.shape[0]*split_ratio):,:]["Survived"].values
y_test = data.iloc[:int(data.shape[0]*split_ratio),:]["Survived"].values

In [19]:
def get_interpret(num_classes):
        def parity(x, num_classes=num_classes):
            return '{:b}'.format(x).count('1') % num_classes
        return parity

In [25]:
def parity(x):
    return '{:b}'.format(x).count('1') % 2

In [32]:
quantum_instance = QuantumInstance(Aer.get_backend('statevector_simulator'), shots=100)
optimizer = COBYLA(maxiter=100)
feature_map = ZZFeatureMap(feature_dimension=X_train.shape[1], reps=1)
ansatz = RealAmplitudes(num_qubits=feature_map._num_qubits, reps=1)

qc = QuantumCircuit(feature_map._num_qubits)
qc.compose(feature_map, inplace=True)
qc.compose(ansatz, inplace=True)

qnn = CircuitQNN(circuit= qc, 
                input_params=feature_map.parameters, 
                weight_params=ansatz.parameters, 
                sparse=False, 
                sampling=False, 
                interpret=parity, 
                output_shape=len(np.unique(y_train, axis=0)), 
                gradient=None, 
                quantum_instance=quantum_instance)

cc = NeuralNetworkClassifier(neural_network=qnn,
                                optimizer=optimizer)

In [33]:
cc.fit(X_train, y_train)

In [34]:
# Model accuracy
acc_train = cc.score(X_train, y_train)
acc_test = cc.score(X_test, y_test)
print("Accuracy on training dataset: {}.".format(acc_train))
print("Accuracy on testing dataset: {}.".format(acc_test))

Accuracy on training dataset: 0.5469845722300141.
Accuracy on testing dataset: 0.5786516853932584.
